## ФАЗОВЫЕ ПОРТРЕТЫ ДИНАМИЧЕСКИХ СИСТЕМ

https://habr.com/ru/post/428984/

Решения обыкновенных дифференциальных уравнений часто удобнее изображать не в привычном виде $y^1(t),...,y^L(t)$, а в *фазовом пространстве*, по осям которого откладываются значения каждой из найденных функций. При этом аргумент t входит в графики лишь параметрически. В случае двух ОДУ такой график – фазовый портрет системы, является кривой на фазовой плоскости и поэтому особенно нагляден.

### Осциллятор

Динамика гармонического осциллятора с затуханием $\gamma$ описывается следующей системой уравнений:

$$
\dot{x}=y\\
\dot{y}=-\omega^2x+\gamma y
$$

и независимо от начальных условий  (х0, у0), приходит в состояние равновесия, точку (0,0) с нулевым углом отклонения и нулевой скоростью.
При $\gamma=0$ решение принимает вид свойственный для классического осциллятора.

In [ ]:
using DifferentialEquations

In [ ]:
using Gadfly
# using Plots
# plotly()

In [ ]:
function garmosc(ω = pi, γ = 0, x0 = 0.1, y0 = 0.1)

    A = [0 1
        -ω^2 γ]

    syst(u,p,t) = A * u # ODE system

    u0 = [x0, y0] # start cond-ns
    tspan = (0.0, 4pi) # time period

    prob = ODEProblem(syst, u0, tspan) # problem to solve
    sol = solve(prob, RK4(),reltol=1e-6, timeseries_steps = 4)

    N = length(sol.u)
    J = length(sol.u[1])

    U = zeros(N, J)

    for i in 1:N, j in 1:J
        U[i,j] = sol.u[i][j] # перепишем ответы в матрицу поудобней
    end
    sol.t, U
end

T, Ans0 = garmosc()

p1 = plot(x = Ans0[:,1], y = Ans0[:,2], Geom.point)
l1 = layer( x = T, y = Ans0[:,1], Geom.point, Theme(default_color="red") )
l2 = layer( x = T, y = Ans0[:,2], Geom.point, Theme(default_color="green") )
p2 = plot(l1, l2)
vstack(p1, p2)

In [ ]:
function garmosc(ω = pi, γ = 0, x0 = 0.1, y0 = 0.1)

    A = [0 1
        -ω^2 γ]

    syst(u,p,t) = A * u # ODE system

    u0 = [x0, y0] # start cond-ns
    tspan = (0.0, 2pi) # time period

    prob = ODEProblem(syst, u0, tspan) # problem to solve
    sol = solve(prob, RK4(),reltol=1e-6, timeseries_steps = 4)

    N = length(sol.u)
    J = length(sol.u[1])

    U = zeros(N, J)

    for i in 1:N, j in 1:J
        U[i,j] = sol.u[i][j] # перепишем ответы в матрицу поудобней
    end
    U
end

set_default_plot_size(10cm, 10cm)

Ans0 = garmosc()
plot(x = Ans0[:,1], y = Ans0[:,2])

Поварьируем частоту, параметр затухания, начальную координату и начальную скорость.

In [ ]:
L = Array{Any}(undef, 3)# создали массив чего угодно (для слоев графиков)
clr = ["red" "green" "yellow"]

function ploter(Answ)
    for i = 1:3
        L[i] = layer(x = Answ[i][:,1], y = Answ[i][:,2], Geom.path, Theme(default_color=clr[i]) )
    end
    p = plot(L[1], L[2], L[3])
end


Ans1 = garmosc.( [pi 2pi 3pi] )# частоту
p1 = ploter(Ans1)

Ans2 = garmosc.( pi, [0.1 0.3 0.5] )# затухание
p2 = ploter(Ans2)

Ans3 = garmosc.( pi, 0.1, [0.2 0.5 0.8] ) 
p3 = ploter(Ans3)

Ans4 = garmosc.( pi, 0.1, 0.1, [0.2 0.5 0.8] )
p4 = ploter(Ans4)

set_default_plot_size(16cm, 16cm)

vstack( hstack(p1,p2), hstack(p3,p4) ) # слепим зоризонтально и вертикально

Теперь рассмотрим не малые колебания, а колебания произвольной амплитуды:

$$
\dot{x}=y\\
\dot{y}=-sin(\omega x)+\gamma y
$$

График решения на фазовой плоскости не является эллипсом (что говорит о нелинейности осциллятора). Чем меньше мы будем выбирать амплитуду колебаний (т.е. начальные условия), тем меньше будет проявляться нелинейность (поэтому-то малые колебания маятника можно считать гармоническими). 

In [ ]:
function ungarmosc(ω = pi, γ = 0, x0 = 0.1, y0 = 0.1)

    function syst(du,u,p,t)
        du[1] = u[2]
        du[2] = -sin( ω*u[1] )+γ*u[2]
    end
    
    u0 = [x0, y0] # start cond-ns
    tspan = (0.0, 2pi) # time period

    prob = ODEProblem(syst, u0, tspan) # problem to solve
    sol = solve(prob, RK4(),reltol=1e-6, timeseries_steps = 4)

    N = length(sol.u)
    J = length(sol.u[1])

    U = zeros(N, J)

    for i in 1:N, j in 1:J
        U[i,j] = sol.u[i][j] # перепишем ответы в матрицу поудобней
    end
    U
end

In [ ]:
Ans1 = ungarmosc.( [pi 2pi 3pi] )
p1 = ploter(Ans1)

Ans2 = ungarmosc.( pi, [0.1 0.4 0.7] )
p2 = ploter(Ans2)

Ans3 = ungarmosc.( pi, 0.1, [0.1 0.5 0.9] )
p3 = ploter(Ans3)

Ans4 = ungarmosc.( pi, 0.1, 0.1, [0.1 0.5 0.9] )
p4 = ploter(Ans4)

vstack( hstack(p1,p2), hstack(p3,p4) )

### Брюсселятор

Эта модель описывает некоторую автокаталитическую химическую реакцию, в которой определенную роль играет диффузия. Модель была предложена в 1968 г. Лефевром и Пригожиным

$$
\dot{u}_1=-(\mu+1)u_1+u^2_1u_2+1 \\
\dot{u}_2=\mu u_1-u^2_1u_2
$$

Неизвестные функции $u_1(t),\,u_2(t)$ отражают динамику концентрации промежуточных продуктов химической реакции. Параметр модели $\mu$ имеет смысл исходной концентрации катализатора (третьего вещества). 

Более детально, эволюцию фазового портрета брюсселятора можно наблюдать, проводя расчеты с различным параметром $\mu$. При его увеличении узел будет сначала постепенно смещаться в точку с координатами (1, $\mu$), пока не достигнет *бифуркационного* значения $\mu$=2. В этой точке происходит качественная перестройка портрета, выражающаяся в рождении предельного цикла. При дальнейшем увеличении $\mu$ происходит лишь количественное изменение параметров этого цикла.

In [ ]:
function brusselator(μ = 0.1, u0 = [0.1; 0.1])
    
    function syst(du,u,p,t)
        du[1] = -(μ+1)*u[1] + u[1]*u[1]*u[2] + 1
        du[2] =      μ*u[1] - u[1]*u[1]*u[2]
    end

    tspan = (0.0, 10) # time period

    prob = ODEProblem(syst, u0, tspan) # problem to solve
    sol = solve(prob, RK4(),reltol=1e-6, timeseries_steps = 1)

    N = length(sol.u)
    J = length(sol.u[1])

    U = zeros(N, J)

    for i in 1:N, j in 1:J
        U[i,j] = sol.u[i][j] # перепишем ответы в матрицу поудобней
    end
    U
end

In [ ]:
L = Array{Any}(undef, 10)

function ploter(Answ)
    for i = 1:length(Answ)
        L[i] = layer(x = Answ[i][:,1], y = Answ[i][:,2], Geom.path )
    end
    plot(L[1], L[2], L[3], L[4], L[5], L[6], L[7], L[8], L[9], L[10])
end

SC = [ [0 0.5], [0 1.5], [2.5 0], [1.5 0], [0.5 1], [1 0], [1 1], [1.5 2], [0.1 0.1], [0.5 0.2] ]

Ans1 = brusselator.( 0.1, SC )
Ans2 = brusselator.( 0.8, SC )
Ans3 = brusselator.( 1.6, SC )
Ans4 = brusselator.( 2.5, SC )
p1 = ploter(Ans1)
p2 = ploter(Ans2)
p3 = ploter(Ans3)
p4 = ploter(Ans4)

set_default_plot_size(16cm, 16cm)

vstack( hstack(p1,p2), hstack(p3,p4) )

### Модель Вольтера «хищник-жертва»

Модель взаимодействия "хищник-жертва" независимо предложили в 1925-1927 гг. Лотка и Вольтерра. Два дифференциальных уравнения 

$$
\dot{x}=A_1x-D_{12}xy \\
\dot{y}=-D_2y+A_{21}xy
$$

моделируют временную динамику численности двух биологических популяций жертвы x  и хищника y. Предполагается, что жертвы размножаются с постоянной скоростью $A_1$, а их численность убывает вследствие поедания хищниками. Хищники же размножаются со скоростью, пропорциональной количеству пищи (с коэффициентом $A_{12}$) и умирают естественным образом (смертность определяется константой $D_2$). Модель замечательна тем, что в такой системе наблюдаются циклическое увеличение и уменьшение численности и хищника, и жертвы, так часто наблюдаемое в природе.

In [ ]:
function predatorvictim(A1 = 0.1, D2 = 1., D12 = 0.1, A21 = 0.1, u0 = [10. 4.] ) # слабое место - целые
    
    function syst1(du,u,p,t)
        du[1] =  A1*u[1] - D12*u[1]*u[2]
        du[2] = -D2*u[2] + A21*u[1]*u[2]
    end

    tspan = (0.0, 100) # time period

    prob = ODEProblem(syst1, u0, tspan) # problem to solve
    sol = solve(prob, RK4(),reltol=1e-6, timeseries_steps = 6)

    N = length(sol.u)
    J = length(sol.u[1])

    U = zeros(N, J)

    for i in 1:N, j in 1:J
        U[i,j] = sol.u[i][j] # перепишем ответы в матрицу по-удобней
    end
    U
end

In [ ]:
L = Array{Any}(undef, 3)
clr = ["green" "yellow" "magenta"] # значения по-возрастанию: зеленый, желтый, розовый

function ploter(Answ)
    for i = 1:length(Answ)
        L[i] = layer(x = Answ[i][:,1], y = Answ[i][:,2], Geom.path, Theme(default_color=clr[i]) )
    end
    plot( L[1], L[2], L[3] )
end

SC = [ [10 8], [10 4], [10 1.5] ]

Ans5 = predatorvictim.( 0.1, 1., 0.1, 0.1, SC )
Ans1 = predatorvictim.( [0.1 0.3 0.6])
Ans2 = predatorvictim.( 0.1, [0.5 1.0 2.0] )
Ans3 = predatorvictim.( 0.1, 1., [0.1 0.3 0.7])
Ans4 = predatorvictim.( 0.1, 1., 0.1, [0.1 0.3 0.7] )
p1 = ploter(Ans1)
p2 = ploter(Ans2)
p3 = ploter(Ans3)
p4 = ploter(Ans4)
p5 = ploter(Ans5)

set_default_plot_size(20cm, 25cm)

vstack( hstack(p1,p2), hstack(p3,p4), p5 )

Несложно убедиться, решая уравнения при различных начальных условиях, что фазовый портрет этой системы представляет собой концентрические замкнутые кривые, окружающие одну стационарную точку типа *центр*. Как видно, модельные колебания численности обеих популяций существенно зависят от начальных условий - после каждого периода колебаний  система возвращается  в ту же точку.

Для учета внутревидовой конкуренции добавим слагаемое $-Bx^2$

$$
\dot{x}=A_1x-D_{12}xy-Bx^2 \\
\dot{y}=-D_2y+A_{21}xy
$$


In [ ]:
function predatorvictim2( B = 1e-7, u0 = [10. 4.], A1 = 0.1, D2 = 1., D12 = 0.1, A21 = 0.1 ) # слабое место - целые
    
    function syst2(du,u,p,t)
        du[1] =  A1*u[1] - D12*u[1]*u[2] - B*u[1]*u[1]
        du[2] = -D2*u[2] + A21*u[1]*u[2] 
    end

    tspan = (0.0, 100) # time period

    prob = ODEProblem(syst2, u0, tspan) # problem to solve
    sol = solve(prob, RK4(),reltol=1e-6, timeseries_steps = 6)

    N = length(sol.u)
    J = length(sol.u[1])

    U = zeros(N, J)

    for i in 1:N, j in 1:J
        U[i,j] = sol.u[i][j] # перепишем ответы в матрицу по-удобней
    end
    U
end

In [ ]:
Ans1 = predatorvictim2.( 1e-3, SC )
Ans2 = predatorvictim2.( [1e-2 5e-3 1e-5] )

p1 = ploter(Ans1)
p2 = ploter(Ans2)

set_default_plot_size(15cm, 8cm)

hstack(p1,p2)

Фазовый портрет системы, имеет одну стационарную точку (аттрактор), на которую «накручивается» решение. В теории динамических систем аттрактор такого типа называется *фокусом*.

К сожалению, попытка введения в модель «хищник-жертва» конкурентной борьбы среди жертв приводит к колебаниям, но, увы, быстро затухающим, что в реальных биологических сообществах не наблюдается.